# KBG Syndrome (Variants in ANKRD11)

This notebook extracts data from [Martinez-Cayuelas E, et al. (2022) Clinical description, molecular delineation and genotype-phenotype correlation in 340 patients with KBG syndrome: addition of 67 new patients](https://pubmed.ncbi.nlm.nih.gov/36446582/).

Accoding to the authors, All variants were referred to the genomic construct hg19 and ANKRD11 transcript NM_013275.6.

There appears to be an error with the variant shown for individual KBG42 (VariantValidator says: "NM_013275.6:c.227G>A: Variant reference (G) does not agree with reference sequence (A)"). Therefore, we remove this row from the analysis.

Note also that c.5483G>T;p.Ser1828* (Parenti2016_P1) should be c.5483C>A (according to the chromatogram in PMID:25652421). We changed this in the excel file.

The authors do not provide all phenotypic data reported in the original publication, but in some cases provide summaries.

In [1]:
import phenopackets as PPKt
from google.protobuf.json_format import MessageToDict, MessageToJson, Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from IPython.display import HTML, display
from pyphetools.creation import *
from pyphetools.validation import *
from pyphetools.visualization import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.8.27


In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
PMID = "PMID:36446582"
title = "Clinical description, molecular delineation and genotype-phenotype correlation in 340 patients with KBG syndrome: addition of 67 new patients"
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", pmid=PMID, pubmed_title=title)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2023-10-09


In [3]:
df = pd.read_excel("input/Martinez-KBG-SupplTable-340-v2.xlsx");
# Drop the row with the erroneous variant as mentioned above
df = df.drop(df[df['Patient ID'] == "KBG42"].index)
df.head()

,Patient origin (1=our cohort; 2=literature),Patient ID,Gender (1=male; 2=female),ID,ASD,ADHD,History of developmental delay (motor and language >18m),ID/ADHD/ASD,Macrodontia and/or other dental anomalies,"Characteristic nose (anteverted, bulbous, and/or prominent)",...,Postnatal short stature <p10,N comorbidities (N=7),Hearing Loss and/or otitis media,"Other comorbidities (seizures, cardiopathy, visual, feeding, cryptorchydism)",Phenotypic score (columns F:P),Variant type (SNV vs CNV),Exon 9 (Yes/No),c.1903_1907del;p.Lys635GInfs*26 (Yes/No),Deletion size (CNV),Variant
0,1,KBG1,1.0,Yes,NaN,NaN,Yes,Yes,NaN,NaN,...,NaN,3.0,NaN,Yes,8,CNV,CNV,CNV,17000,16q24.3(89336307_89354085)x1
1,1,KBG2,1.0,Yes,NaN,NaN,No,Yes,NaN,NaN,...,NaN,1.0,NaN,Yes,4,CNV,CNV,CNV,250000,16q24.3(89256478_89506223)x1
2,1,KBG3,2.0,NaN,NaN,NaN,Yes,NaN,NaN,NaN,...,NaN,3.0,Yes,Yes,5,SNV,Yes,No,SNV,c.2398_2401del;p.Glu800Asnfs*62
3,1,KBG4,2.0,Yes,NaN,NaN,Yes,Yes,NaN,No,...,No,1.0,No,Yes,6,SNV,Yes,No,SNV,c.7083del;p.Thr2362Profs*39
4,1,KBG5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,SNV,Yes,Yes,SNV,c.1903_1907del;p.Lys635GInfs*26


In [4]:
column_mapper_d = {}

items = {
    'ID':['Intellectual disability','HP:0001249'],
    'ASD':['Autistic behavior','HP:0000729'],
    'ADHD':['Attention deficit hyperactivity disorder', 'HP:0007018'],
    'History of developmental delay (motor and language >18m)': ['Global developmental delay', 'HP:0001263'],
    'ID/ADHD/ASD':['Abnormality of mental function', 'HP:0011446'],
    'Macrodontia and/or other dental anomalies':['Macrodontia', 'HP:0001572'],
    'Characteristic nose (anteverted, bulbous, and/or prominent)': ['Abnormal external nose morphology', 'HP:0010938'],
    'Triangular face': ['Triangular face', 'HP:0000325'],
    'Characteristic eyebrows': ['Thick eyebrow', 'HP:0000574'],
    'Long philtrum': ['Long philtrum', 'HP:0000343'],
    'Characteristic ears (large, prominent, and/or low-set)': ['Abnormality of the outer ear', 'HP:0000356'],
    'Hand anomalies (brachydactyly or clinodactyly)': ['Abnormality of the hand', 'HP:0001155'],
    'Postnatal short stature <p10': ['Short stature', 'HP:0004322'],
    'Hearing Loss and/or otitis media': ['Hearing impairment', 'HP:0000365']
}

item_column_mapper_d = hpo_cr.initialize_simple_column_maps(column_name_to_hpo_label_map=items, observed='Yes', excluded='No', non_measured='NA')

for k, v in item_column_mapper_d.items():
    column_mapper_d[k] = v
print(f"We created {len(column_mapper_d)} simple column mappers")


We created 14 simple column mappers


In [5]:
def get_hgvs_if_possible_otherwise_cnv(value):
    if ";" in value:
        fields = value.split(";")
        return f"SNV;{fields[0]}"
    else:
        return f"CNV;{value}"

df["ANKRD11_variant"] = df["Variant"].apply(lambda x: get_hgvs_if_possible_otherwise_cnv(x))
df.head()

,Patient origin (1=our cohort; 2=literature),Patient ID,Gender (1=male; 2=female),ID,ASD,ADHD,History of developmental delay (motor and language >18m),ID/ADHD/ASD,Macrodontia and/or other dental anomalies,"Characteristic nose (anteverted, bulbous, and/or prominent)",...,N comorbidities (N=7),Hearing Loss and/or otitis media,"Other comorbidities (seizures, cardiopathy, visual, feeding, cryptorchydism)",Phenotypic score (columns F:P),Variant type (SNV vs CNV),Exon 9 (Yes/No),c.1903_1907del;p.Lys635GInfs*26 (Yes/No),Deletion size (CNV),Variant,ANKRD11_variant
0,1,KBG1,1.0,Yes,NaN,NaN,Yes,Yes,NaN,NaN,...,3.0,NaN,Yes,8,CNV,CNV,CNV,17000,16q24.3(89336307_89354085)x1,CNV;16q24.3(89336307_89354085)x1
1,1,KBG2,1.0,Yes,NaN,NaN,No,Yes,NaN,NaN,...,1.0,NaN,Yes,4,CNV,CNV,CNV,250000,16q24.3(89256478_89506223)x1,CNV;16q24.3(89256478_89506223)x1
2,1,KBG3,2.0,NaN,NaN,NaN,Yes,NaN,NaN,NaN,...,3.0,Yes,Yes,5,SNV,Yes,No,SNV,c.2398_2401del;p.Glu800Asnfs*62,SNV;c.2398_2401del
3,1,KBG4,2.0,Yes,NaN,NaN,Yes,Yes,NaN,No,...,1.0,No,Yes,6,SNV,Yes,No,SNV,c.7083del;p.Thr2362Profs*39,SNV;c.7083del
4,1,KBG5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,SNV,Yes,Yes,SNV,c.1903_1907del;p.Lys635GInfs*26,SNV;c.1903_1907del


In [6]:
ANKRD11_transcript = 'NM_013275.6'
vvalidator = VariantValidator(genome_build="hg38", transcript=ANKRD11_transcript)
var_d = {}
for item in df["ANKRD11_variant"].unique():
    fields = item.split(";")
    if fields[0] == "SNV":
        v = fields[1]
        var = vvalidator.encode_hgvs(v)
        var_d[item] = var
    elif fields[0] == "CNV":
        cnv = fields[1] + " (hg19)"
        if "x1" in cnv:
            var = StructuralVariant.chromosomal_deletion(cell_contents=cnv, gene_symbol="ANKRD11", gene_id="HGNC:21316")
        elif "x3" in cnv:
            var = StructuralVariant.chromosomal_duplication(cell_contents=cnv, gene_symbol="ANKRD11", gene_id="HGNC:21316")
        else:
            raise ValueError(f"Unrecognized cnv type: {cnv}")
        var_d[item] = var
print(f"Extracted {len(var_d)} distinct variants")

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.2398_2401del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.7083del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.1903_1907del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.7407C>G/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.6691dup/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.3590_3594del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.6792dup/NM_013275.6?content-type=application%2Fjso

In [7]:
varMapper = VariantColumnMapper(variant_d=var_d,
                                variant_column_name="ANKRD11_variant",
                                default_genotype="heterozygous"
                                )

In [8]:
sexMapper = SexColumnMapper(male_symbol='1.0', female_symbol='2.0', unknown_symbol='nan', column_name='Gender (1=male; 2=female)')
ageMapper = AgeColumnMapper.by_year('Patient origin (1=our cohort; 2=literature)')
#sexMapper.preview_column(df['Gender (1=male; 2=female)'])

In [9]:
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr,
                        column_mapper_d=column_mapper_d, 
                        individual_column_name="Patient ID",
                        sexmapper=sexMapper,
                        agemapper=ageMapper,
                        variant_mapper=varMapper,
                        metadata=metadata,
                        pmid=PMID)
kbg = Disease(disease_id='OMIM:148050', disease_label='KBG syndrome')
encoder.set_disease(kbg)

In [10]:
individuals = encoder.get_individuals()
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1)
qc = QcVisualizer(ontology=hpo_ontology, cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
ERROR,INSUFFICIENT_HPOS,11
WARNING,REDUNDANT,224
INFORMATION,NOT_MEASURED,2321


In [11]:
individuals = cvalidator.get_error_free_individual_list()
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1)
qc = QcVisualizer(ontology=hpo_ontology, cohort_validator=cvalidator)
display(HTML(qc.to_html()))

In [12]:
phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(individual_list==phenopackets)
display(HTML(table.to_html()))

TypeError: PhenopacketTable.__init__() got an unexpected keyword argument 'phenopacket_list'

In [15]:
output_directory = "phenopackets"
Individual.output_individuals_as_phenopackets(individual_list=individuals, 
                                             metadata=metadata,
                                             outdir=output_directory)

We output 328 GA4GH phenopackets to the directory phenopackets


In [ ]:
# pxf validate --hpo hp.json *.json
# no errors